# Summary
Given a trade represented by the ticker symbol, date and match_id, get the trade's exchange_timestamp and enrich with BBO (from certain venues) and NBBO just before the trade 

Add the necessary imports

In [23]:
import datetime
import hpq
import ijson
import numpy

A trade denoted by the security, match_id and approximate time of the trade is received. All times in UTC.

In [49]:
match_id_of_trade = 52983542269494
security = "XOM"
apprx_date_time_of_trade = datetime.datetime(2023, 1, 10, 17, 10, 50, 40400)
print(apprx_date_time_of_trade)

2023-01-10 17:10:50.040400


Query for trades starting from 15 minutes before the approximate trade time to the approximate trade time. Identify the trade using match_id and store the exchange_timestamp of the trade

In [50]:
connection = hpq.create_web_socket_client()
request = {
    "query": "ticks",
    "messages": "TRD",
    "source": "sip",
    "product": security,
    "time_zone": "UTC",
    "date": apprx_date_time_of_trade.strftime("%Y-%m-%d"),
    "start_time": (apprx_date_time_of_trade - datetime.timedelta(minutes=15)).strftime(
        "%H:%M:%S.%f"
    ),
    "end_time": apprx_date_time_of_trade.strftime("%H:%M:%S.%f"),
}

trades = ijson.items(connection.stream(request), "item")
trade_exch_timestamp = 0
for item in trades:
    if item["match_id"] == match_id_of_trade:
        print(item)
        trade_exch_timestamp = item["exchange_timestamp"]

{'name': 'XOM', 'type': 'trade', 'receipt_timestamp': 1673370040455565311, 'exchange_timestamp': 1673370040455334144, 'sequence_number': 979791, 'price': Decimal('108.63'), 'quantity': 100, 'match_id': 52983542269494, 'printable': True, 'sale_condition': 'RegularTrade', 'sale_condition2': 'NoSaleCondition', 'sale_condition3': 'NoSaleCondition', 'sale_condition4': 'NoSaleCondition', 'trade_through_exempt_indicator': 'NotATradeThroughExemption', 'market_participant': 'NYSE', 'session': 11, 'feed': 'cts', 'session_ip_addr_a': '224.0.89.11', 'session_ip_port_a': 40011, 'session_ip_addr_b': '224.0.89.139', 'session_ip_port_b': 40011}


Request a snapshot of bbo_quotes from 1 nanosecond before the trade's exchange_timestamp

In [51]:
trade_exch_timestamp -= 1
trade_date_time = numpy.datetime64(trade_exch_timestamp, "ns")

snap_request = {
    "query": "snapshot",
    "type": "bbo_quotes",
    "source": "sip[nyse,nysearca,bats,nasdaqomx]",
    "product": "XOM",
    "time_zone": "UTC",
    "date": numpy.datetime_as_string(trade_date_time, "D", "UTC"),
    "time": trade_date_time.astype(str)[11:29],
}

snapshot = ijson.items(connection.stream(snap_request), "item")
for item in snapshot:
    print(item["market_participant"])
    print("  bid_px", item["bid_price"])
    print("  bid_qty", item["bid_quantity"])
    print("  ask_px", item["ask_price"])
    print("  ask_qty", item["ask_quantity"])

1673370040455334144
2023-01-10T17:00:40.455334143
NASDAQOMX
  bid_px 108.63
  bid_qty 100
  ask_px 108.66
  ask_qty 200
BATS
  bid_px 108.62
  bid_qty 200
  ask_px 108.65
  ask_qty 100
NYSE
  bid_px 108.63
  bid_qty 100
  ask_px 108.64
  ask_qty 200
NYSEArca
  bid_px 108.62
  bid_qty 100
  ask_px 108.66
  ask_qty 200


Request a snapshot of the nbbo quote

In [45]:
snap_request["type"] = "nbbo_quotes"
snap_request["source"] = "sip"
nbbo_snapshot = ijson.items(connection.stream(snap_request), "item")
print("NBBO")
for item in nbbo_snapshot:
    print("  bid_px", item["bid_price"])
    print("  bid_qty", item["bid_quantity"])
    print("  ask_px", item["ask_price"])
    print("  ask_qty", item["ask_quantity"])

NBBO
  bid_px 108.63
  bid_qty 500
  ask_px 108.64
  ask_qty 300
